# 1.2.0 Build Final Dataset

### Methodology:
The main goal of this stage is to compile the final dataset that merges the target variable with features generated from previous steps, specifically those created by build_previous_internal_app_features and build_aggregate_credit_report_information_by. This comprehensive dataset will serve as the foundation for all subsequent modeling efforts, such as training, and validation machine learning models aimed at predicting customer creditworthiness.

Data Integration:
1. Merge Target Data: Start by merging the dataset containing the target variable (e.g., loan default status) with features derived from the build_previous_internal_app_features function, which provides insights into the customer's past interactions with credit products.
2. Add Aggregated Credit Information: Integrate additional features from the build_aggregate_credit_report_information_by function. This includes detailed credit behavior metrics at the customer level, further enriching the dataset.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def build_previous_internal_app_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Processes a Main Dataset to create features derived from the customer's history within the organization.
    These features include:
        - Ratio of approved BNPL applications to total BNPL applications.
        - Days between the last BNPL application and the credit application date.
        - Days between the first BNPL application and the credit application date.
        - Days from account creation to credit application.
        - Total counts of SF and BNPL applications, including approved BNPL applications.
        - Number of inquiries to credit reports from external entities in the last 3 and 6 months.
   

    Parameters:
    df (pd.DataFrame): A DataFrame containing the main dataset with the customer's history within the organization.

    Returns:
    pd.DataFrame: A DataFrame containing the loan ID and the newly created features prefixed with 'previous_internal_apps__'.
    """
    
    df.columns = [i.lower() for i in df.columns]
    
    df = df.assign(
        previous_internal_apps__ratio_bnpl_approved=(df["n_bnpl_approved_apps"]/df["n_bnpl_apps"]).fillna(0),
        previous_internal_apps__last_bnpl_app_to_application_days=(df["application_datetime"] - df["first_bnpl_app_date"]).dt.days,
        previous_internal_apps__first_bnpl_app_to_application_days=(df["application_datetime"] - df["last_bnpl_app_date"]).dt.days,
        previous_internal_apps__account_to_application_days=df["account_to_application_days"],
        previous_internal_apps__n_sf_apps=df["n_sf_apps"].fillna(0),
        previous_internal_apps__n_bnpl_apps=df["n_bnpl_apps"].fillna(0),
        previous_internal_apps__n_bnpl_approved_apps=df["n_bnpl_approved_apps"].fillna(0),
        previous_internal_apps__n_inquiries_l3m=df["n_inquiries_l3m"].fillna(0),
        previous_internal_apps__n_inquiries_l6m=df["n_inquiries_l6m"].fillna(0),
    )

    features = [i for i in df.columns if "previous_internal_apps__" in i]
    
    return df[["loan_id"] + features]


def build_aggregate_credit_report_information_by(df: pd.DataFrame, aggregate_by: str) -> pd.DataFrame:
    
    """
    Aggregates credit report information by customer ID and a specified category, computing various 
    statistical measures for each group. This function creates a wide-format DataFrame where each row 
    represents a unique customer and columns represent aggregated metrics of credit-related activities 
    across different categories specified by 'aggregate_by'.

    Parameters:
    - df: The input DataFrame containing credit report data.
    - aggregate_by: The column name by which to further group the data (e.g., 'account_type').

    Returns:
    - pd.DataFrame: A pivot table where index is 'customer_id', columns are created by the values of 
      'aggregate_by', and cells contain aggregated credit report metrics such as sums, medians, and 
      standard deviations of financial metrics. Each feature is prefixed with 'credit_reports__' to 
      denote its origin from credit report data.

    Example of aggregated metrics include:
    - Count of inquiries
    - Sum, median, and standard deviation of maximum credit
    - Number of unique credit types
    - Maximum, median, and standard deviation of delayed payment severity
    """
    
    df_aggregates = df.groupby(["customer_id", aggregate_by]).agg({
        "cdc_inquiry_id": ["count"],
        "max_credit": ['sum', 'median', 'std'],
        "credit_limit": ['sum', 'median', 'std'],
        "current_balance": ['sum', 'median', 'std'],
        "balance_due_worst_delay": ['max', 'median', 'std'],
        "balance_due": ['sum', 'median', 'std'],
        "debt_ratio": ['max', 'median', 'std'],
        "credit_type": ["nunique"], 
        "business_type": ["nunique"],
        "age": ['max', 'median', 'std'],      
        "severity_delayed_payments":  ['max', 'median', 'std'],
        "balance_due_ratio":  ['max', 'median', 'std'],
        "balance_due_worst_delay_ratio":  ['max', 'median', 'std'],
        "has_delayed_payments":  ['sum'],
        "is_individual_responsibility":  ['sum'],
        "payment_amount": ['sum']
    })
    df_aggregates.columns = ["_".join(i) for i in df_aggregates.columns.values]
    df_aggregates = df_aggregates.reset_index()

    values = df_aggregates.columns.to_list()
    values.remove("customer_id")
    values.remove(aggregate_by)
    
    df_pivot = df_aggregates.pivot_table(
        index='customer_id',
        columns=aggregate_by,
        values=values,
        aggfunc='first'
    )
    
    features = ["credit_reports__" + "_".join(col).lower() for col in df_pivot.columns.values]
    df_pivot.columns = features

    return df_pivot.reset_index()


def build_aggregate_credit_report_information(df: pd.DataFrame, aggregate_column_names: str = "credit_reports__") -> pd.DataFrame:
    """
    Aggregates key financial indicators from a credit report dataset at the customer level. This function
    computes various statistical metrics such as count, sum, max, min, mean, median, and standard deviation
    for different financial variables to provide a comprehensive summary of each customer's credit activities.

    Parameters:
    - df: The DataFrame containing credit report data with multiple entries per customer.
    - aggregate_column_names: A prefix for the column names in the aggregated DataFrame,
      helping to identify the source of the features. Defaults to 'credit_reports__'.

    Returns:
    - pd.DataFrame: A DataFrame where each row corresponds to a unique customer_id and columns represent
      aggregated metrics for various credit-related features. Column names are prefixed with the value 
      provided in `aggregate_column_names`, followed by the specific aggregation type (e.g., 'sum', 'max').

    Aggregates the following metrics for each customer:
    - Count of credit inquiries
    - Sum of maximum credit extended
    - Sum of credit limits across all accounts
    - Sum of current balances across accounts
    - Maximum and sum of balances due
    - Maximum, median, mean, and standard deviation of the debt ratio
    - Number of unique credit types and business types utilized by the customer
    - Maximum and minimum age of accounts
    - Maximum, median, mean, and standard deviation of severity of delayed payments
    - Aggregated metrics related to balance due ratios
    - Sum of instances where payments were delayed
    - Sum of instances denoting individual responsibility for the credit
    - Sum of payment amounts
    """
    
    df_aggregates = df.groupby(["customer_id"]).agg({
        "cdc_inquiry_id": ["count"],
        "max_credit": ["sum"],
        "credit_limit": ["sum"],
        "current_balance": ["sum"],
        "balance_due_worst_delay": ['max'],
        "balance_due": ['sum'],
        "debt_ratio": ['max', 'median', 'mean', 'std'],
        "credit_type": ["nunique"],  
        "business_type": ["nunique"], 
        "age": ['max', 'min'],     
        "severity_delayed_payments": ['max', 'median', 'mean', 'std'],
        "balance_due_ratio":  ['max', 'median', 'mean', 'std'],
        "balance_due_worst_delay_ratio":  ['max', 'median', 'mean', 'std'],
        "has_delayed_payments":  ['sum'],
        "is_individual_responsibility": ['sum'],
        "payment_amount": ['sum']
    })
    df_aggregates.columns = [aggregate_column_names + "_".join(i) for i in df_aggregates.columns.values]
    df_aggregates = df_aggregates.reset_index()

    return df_aggregates


def build_credit_report_features(df_aux: pd.DataFrame) -> pd.DataFrame:
    """
    Processes and enriches a DataFrame containing credit report data by adding derived features,
    aggregating data, and preparing the dataset for further analysis or modeling.

    This function handles:
    - Standardizing column names and data types.
    - Calculating various financial ratios and flags based on credit data.
    - Aggregating credit data at the customer level to provide a holistic view of their credit status.
    - Merging different aggregations to form a comprehensive feature set per customer.

    Parameters:
    - df_aux: The input DataFrame with raw credit report data.

    Returns:
    - pd.DataFrame: A DataFrame indexed by 'customer_id' with new features derived from credit report data,
      including ratios of credit use, payment behaviors, and aggregate metrics of credit activities.
    """
    
    df = df_aux.copy()
    df.columns = [i.lower() for i in df.columns]
    df["account_type"] = df["account_type"].str.replace(" ", "_")
    df = df.astype({"delayed_payments": "float"})
    df[["responsability_type", "credit_type", "business_type"]]
    
    df = df.assign(
        age = np.where(
            df["loan_opening_date"].isnull(), np.nan, np.where(
                df["loan_closing_date"].isnull(), (df["application_datetime"] - df["loan_opening_date"]).dt.days, (df["loan_closing_date"] - df["loan_opening_date"]).dt.days)),
        is_opening = np.where(
            df["loan_closing_date"].isnull(), 1, np.where(~df["loan_closing_date"].isnull(), 0, np.nan)),
        debt_ratio = (df["current_balance"] / df["max_credit"]).replace([np.inf, -np.inf], np.nan),
        severity_delayed_payments = (df["delayed_payments"] / df["total_payments"]).replace([np.inf, -np.inf], np.nan),
        balance_due_ratio = (df["balance_due"] / df["max_credit"]).replace([np.inf, -np.inf], np.nan),
        balance_due_worst_delay_ratio = (df["balance_due_worst_delay"] / df["max_credit"]).replace([np.inf, -np.inf], np.nan),
        has_delayed_payments = np.where(df["delayed_payments"]>0, 1, np.where(df["delayed_payments"]==0, 0, np.nan)),
        is_individual_responsibility = np.where(df["responsability_type"]=="INDIVIDUAL (TITULAR)", 1, np.where(~df["responsability_type"].isnull(), 0, np.nan))
    )

    
    agg_df = build_aggregate_credit_report_information(df).rename(columns={
        "credit_reports__cdc_inquiry_id_count": "credit_reports__loans_count",
        "credit_reports__is_opening_sum": "credit_reports__opening_loan_count",
        "credit_reports__has_delayed_payments_sum": "credit_reports__loans_with_at_least_one_delayed_count",
    })

    df_aux = df[df["is_opening"]==1]
    agg_df_open_loans = build_aggregate_credit_report_information(df_aux, aggregate_column_names="credit_reports__open_loans_").rename(columns={
        "credit_reports__open_loans_cdc_inquiry_id_count": "credit_reports__open_loans_count",
        "credit_reports__open_loans_is_opening_sum": "credit_reports__opening_loan_count",
        "credit_reports__open_loans_has_delayed_payments_sum": "credit_reports__open_loans_with_at_least_one_delayed_count",
    })
    
    agg_df_by_credit_type = build_aggregate_credit_report_information_by(df, aggregate_by="account_type")
    
    df_pivot = df[["customer_id"]].drop_duplicates()
    df_pivot = pd.merge(df_pivot, agg_df, how="left", on="customer_id")
    df_pivot = pd.merge(df_pivot, agg_df_open_loans, how="left", on="customer_id")
    df_pivot = pd.merge(df_pivot, agg_df_by_credit_type, how="left", on="customer_id")

    df_pivot = df_pivot.assign(
        credit_reports__opening_loans_ratio = df_pivot["credit_reports__open_loans_count"]/df_pivot["credit_reports__loans_count"],
        credit_reports__loans_with_at_least_one_delayed_ratio = df_pivot[ "credit_reports__loans_with_at_least_one_delayed_count"]/df_pivot["credit_reports__loans_count"],
        credit_reports__debt_ratio = df_pivot["credit_reports__balance_due_sum"]/df_pivot["credit_reports__max_credit_sum"],
        credit_reports__debt_due_ratio = df_pivot["credit_reports__balance_due_sum"]/df_pivot["credit_reports__balance_due_sum"]
    )

    return df_pivot
    
    

    

In [3]:
DATA_PATH = Path.cwd().parent / "data"
MAIN_DATASET_PATH = DATA_PATH / "raw_data/main_dataset.parquet"
CREDIT_REPORT_DATA_PATH = DATA_PATH / "raw_data/credit_reports.parquet"

### 1. Load Data

In [4]:
df = pd.read_parquet(MAIN_DATASET_PATH)
df["loan_origination_datetime_month"] = df["LOAN_ORIGINATION_DATETIME"].dt.strftime("%Y-%m")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14454 entries, 0 to 14453
Data columns (total 18 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   customer_id                      14454 non-null  int64         
 1   loan_id                          14454 non-null  int64         
 2   ACC_CREATION_DATETIME            14454 non-null  datetime64[us]
 3   APPLICATION_DATETIME             14454 non-null  datetime64[us]
 4   LOAN_ORIGINATION_DATETIME        14454 non-null  datetime64[us]
 5   max_days_late                    14454 non-null  int64         
 6   target                           14454 non-null  int64         
 7   account_to_application_days      14454 non-null  int64         
 8   n_sf_apps                        6806 non-null   float64       
 9   first_app_date                   6806 non-null   datetime64[us]
 10  last_app_date                    6806 non-null   datetime6

In [5]:
credit_report_df = pd.read_parquet(CREDIT_REPORT_DATA_PATH)
credit_report_df = pd.merge(credit_report_df, df[["LOAN_ORIGINATION_DATETIME", "customer_id", "APPLICATION_DATETIME"]], how="left", on="customer_id")

### 2. Build previous_internal_apps_features

In [6]:
df_previous_internal_apps_features = build_previous_internal_app_features(df)
df_previous_internal_apps_features

,loan_id,previous_internal_apps__ratio_bnpl_approved,previous_internal_apps__last_bnpl_app_to_application_days,previous_internal_apps__first_bnpl_app_to_application_days,previous_internal_apps__account_to_application_days,previous_internal_apps__n_sf_apps,previous_internal_apps__n_bnpl_apps,previous_internal_apps__n_bnpl_approved_apps,previous_internal_apps__n_inquiries_l3m,previous_internal_apps__n_inquiries_l6m
0,1,0.0,0.0,0.0,245,1.0,1.0,0.0,91.0,91.0
1,2,0.0,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,37.0,47.0,47,1.0,2.0,0.0,13.0,13.0
4,5,0.0,24.0,24.0,24,1.0,1.0,0.0,21.0,21.0
...,...,...,...,...,...,...,...,...,...,...
14449,14450,0.0,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0
14450,14451,0.0,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0
14451,14452,0.0,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0
14452,14453,0.0,1.0,4.0,692,0.0,3.0,0.0,10.0,10.0


In [7]:
credit_report_df

,customer_id,INQUIRY_TIME,CDC_INQUIRY_ID,INQUIRY_DATE,PREVENTION_KEY,CURRENCY,MAX_CREDIT,CREDIT_LIMIT,PAYMENT_AMOUNT,UPDATE_DATE,...,TOTAL_PAYMENTS,DELAYED_PAYMENTS,CURRENT_PAYMENT,WORST_DELAY,TOTAL_REPORTED_PAYMENTS,CURRENT_BALANCE,BALANCE_DUE,BALANCE_DUE_WORST_DELAY,LOAN_ORIGINATION_DATETIME,APPLICATION_DATETIME
0,4223,2022-04-01 00:32:36.689000+00:00,710278-27993a6e-2885-48d4-a753-9249e7022af1,2022-04-01,SIN OBSERVACION,PESOS MEXICANOS,1244.0,9900.0,1244.0,2022-02-28,...,12.0,7,13,13.0,0,1244.0,1244.0,1244.0,2022-07-07 17:31:33,2022-05-23 07:00:00
1,4223,2022-04-01 00:32:36.689000+00:00,710278-27993a6e-2885-48d4-a753-9249e7022af1,2022-04-01,SIN OBSERVACION,PESOS MEXICANOS,1722.0,9900.0,1722.0,2022-02-28,...,0.0,3,13,13.0,0,1722.0,1722.0,1722.0,2022-07-07 17:31:33,2022-05-23 07:00:00
2,4223,2022-04-01 00:32:36.689000+00:00,710278-27993a6e-2885-48d4-a753-9249e7022af1,2022-04-01,SIN OBSERVACION,PESOS MEXICANOS,6346.0,0.0,482.0,2022-03-28,...,16.0,0,V,0.0,0,943.0,0.0,0.0,2022-07-07 17:31:33,2022-05-23 07:00:00
3,3490,2022-02-15 02:30:22.086000+00:00,622857-6b4e9d95-7491-40c3-bccd-442de7f94c58,2022-02-15,SIN OBSERVACION,PESOS MEXICANOS,11600.0,0.0,232.0,2022-02-01,...,100.0,4,01,1.0,0,6185.0,116.0,116.0,2022-07-02 16:03:31,2022-06-23 07:00:00
4,6486,2022-06-25 01:57:14.868000+00:00,875073-46a5f149-19db-4193-b92a-b8f41fb3e896,2022-06-25,SIN OBSERVACION,PESOS MEXICANOS,2452.0,16800.0,2452.0,2021-09-30,...,0.0,19,13,13.0,0,2452.0,2452.0,2452.0,2022-07-05 11:01:14,2022-06-26 07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287351,12079,2022-12-24 22:21:36.102000+00:00,1053785-35891c8d-97ba-4dce-9f5f-e7bf8684473a,2022-12-24,SIN OBSERVACION,PESOS MEXICANOS,1609.0,1609.0,634.0,2022-12-01,...,3.0,11,01,1.0,0,634.0,634.0,634.0,2023-05-29 12:18:28,2023-05-26 07:00:00
287352,12079,2022-12-24 22:21:36.102000+00:00,1053785-35891c8d-97ba-4dce-9f5f-e7bf8684473a,2022-12-24,SIN OBSERVACION,PESOS MEXICANOS,248.0,248.0,263.0,2022-12-01,...,1.0,11,01,1.0,0,263.0,263.0,263.0,2023-05-29 12:18:28,2023-05-26 07:00:00
287353,12079,2022-12-24 22:21:36.102000+00:00,1053785-35891c8d-97ba-4dce-9f5f-e7bf8684473a,2022-12-24,SIN OBSERVACION,PESOS MEXICANOS,3267.0,3267.0,1243.0,2022-12-01,...,3.0,10,03,3.0,0,3666.0,3666.0,3666.0,2023-05-29 12:18:28,2023-05-26 07:00:00
287354,12079,2022-12-24 22:21:36.102000+00:00,1053785-35891c8d-97ba-4dce-9f5f-e7bf8684473a,2022-12-24,SIN OBSERVACION,PESOS MEXICANOS,496.0,496.0,189.0,2022-12-01,...,3.0,10,03,3.0,0,557.0,557.0,557.0,2023-05-29 12:18:28,2023-05-26 07:00:00


### 3. Build credit_reports_features

In [8]:
df_credit_reports_features = build_credit_report_features(credit_report_df)
df_credit_reports_features["credit_reports__loans_count"] = df_credit_reports_features["credit_reports__loans_count"].fillna(0)
df_credit_reports_features

,customer_id,credit_reports__loans_count,credit_reports__max_credit_sum,credit_reports__credit_limit_sum,credit_reports__current_balance_sum,credit_reports__balance_due_worst_delay_max,credit_reports__balance_due_sum,credit_reports__debt_ratio_max,credit_reports__debt_ratio_median,credit_reports__debt_ratio_mean,...,credit_reports__severity_delayed_payments_std_credito_de_habilitacion_o_avio,credit_reports__severity_delayed_payments_std_hipoteca,credit_reports__severity_delayed_payments_std_pagos_fijos,credit_reports__severity_delayed_payments_std_por_determinar,credit_reports__severity_delayed_payments_std_revolvente,credit_reports__severity_delayed_payments_std_sin_limite_preestablecido,credit_reports__opening_loans_ratio,credit_reports__loans_with_at_least_one_delayed_ratio,credit_reports__debt_ratio,credit_reports__debt_due_ratio
0,4223,3,9312.0,19800.0,3909.0,1722.0,2966.0,1.000000,1.000000,0.716199,...,NaN,NaN,0.412479,NaN,NaN,NaN,1.000000,0.666667,0.318514,1.0
1,3490,1,11600.0,0.0,6185.0,116.0,116.0,0.533190,0.533190,0.533190,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,0.010000,1.0
2,6486,2,2452.0,16800.0,2452.0,2452.0,2452.0,1.000000,1.000000,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.500000,1.000000,1.0
3,4075,52,317915.0,65044.0,35140.0,7880.0,35140.0,3.875391,0.000000,0.120348,...,NaN,NaN,0.158509,NaN,0.000000,NaN,0.076923,0.307692,0.110533,1.0
4,437,48,775025.0,499063.0,13226.0,12666.0,20264.0,1.518111,0.000000,0.070266,...,NaN,NaN,1.476400,NaN,NaN,NaN,0.062500,0.145833,0.026146,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9244,12781,25,152028.0,101914.0,7497.0,2049.0,7497.0,1.000000,0.000000,0.173574,...,NaN,NaN,0.151585,NaN,NaN,NaN,0.200000,0.240000,0.049313,1.0
9245,4428,19,280437.0,221200.0,316611.0,5858.0,17571.0,1.765642,0.000000,0.127265,...,NaN,NaN,0.481125,NaN,NaN,NaN,0.210526,0.263158,0.062656,1.0
9246,10963,2,18141.0,500.0,15217.0,176.0,0.0,0.839545,0.827351,0.827351,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.500000,0.000000,NaN
9247,768,21,155928.0,81978.0,26673.0,11425.0,11425.0,1.000000,0.000000,0.185721,...,NaN,NaN,0.218844,NaN,NaN,NaN,0.285714,0.095238,0.073271,1.0


### 4. Merge Data

In [9]:
final_df = pd.merge(
    df[["customer_id", "loan_id", "target", "loan_origination_datetime_month"]], 
    df_previous_internal_apps_features,
    how="left",
    on="loan_id"
)
final_df.shape

(14454, 13)

In [10]:
final_df = pd.merge(
    final_df, 
    df_credit_reports_features,
    how="left",
    on="customer_id"
)
final_df = final_df.fillna(0)

In [11]:
formatted_date = datetime.now().strftime("%Y%m")

final_df.to_pickle(DATA_PATH / f"processed/{formatted_date}_final_dataset.pickle")